<h1>To Do:</h1>

* Clean Up Notebook

In [44]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import preprocessing 
from sklearn.model_selection import cross_val_score

* https://scikit-learn.org/stable/modules/model_evaluation.html
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html
* https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score

In [45]:
# print(training.columns)
# print(training.head(10))

In [46]:
training = pd.read_csv("train.csv")
training = training.drop(["Unnamed: 0","id"], axis=1)

test = pd.read_csv("test.csv")
training = test.drop(["Unnamed: 0", "id"], axis=1)

In [47]:
le = preprocessing.LabelEncoder()

# training["Gender"] = le.fit_transform(training["Gender"])
# print(training["Gender"].describe())
for i in ['Gender', 'Customer Type', "satisfaction","Class", "Type of Travel"]:
    training[i] = le.fit_transform(training[i])
    test[i] = le.fit_transform(test[i])
print(training["Class"].describe())

count    25976.000000
mean         0.592778
std          0.623703
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          2.000000
Name: Class, dtype: float64


In [48]:
training_x = training.drop(["satisfaction"], axis=1)
training_y = training[["satisfaction"]]
test_x = test.drop(["satisfaction"], axis=1)
test_y = test[["satisfaction"]]

In [49]:
print(training["satisfaction"].describe())
print(test["satisfaction"].describe())

count    25976.000000
mean         0.438982
std          0.496272
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: satisfaction, dtype: float64
count    25976.000000
mean         0.438982
std          0.496272
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: satisfaction, dtype: float64


In [50]:
print(training_y.describe())

       satisfaction
count  25976.000000
mean       0.438982
std        0.496272
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000


In [51]:
print(f"NaN in training set:\n{training.isnull().sum()}")
print(f"NaN in test set:\n{test.isnull().sum()}")

NaN in training set:
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             83
satisfaction                          0
dtype: int64
NaN in test set:
Unnamed: 0                   

<h1> Data Modifications: </h1>

<li> Dropped number column, id </li>

Note that we ensure that id must not occur in both training and testing in order to recdue the likelihood of data leaking.

In [52]:
# print(training_x["Arrival Delay in Minutes"].describe())
training_x["Arrival Delay in Minutes"] = training_x["Arrival Delay in Minutes"].fillna(-9999)
test_x["Arrival Delay in Minutes"] = test_x["Arrival Delay in Minutes"].fillna(-9999)
# print(training_x["Arrival Delay in Minutes"].describe())
print(training.head())
# Change was not significant to modify statistics 

count    25976.000000
mean       -17.255621
std        566.388577
min      -9999.000000
25%          0.000000
50%          0.000000
75%         13.000000
max       1115.000000
Name: Arrival Delay in Minutes, dtype: float64


In [59]:
print(training.describe())

             Gender  Customer Type           Age  Type of Travel  \
count  25976.000000   25976.000000  25976.000000    25976.000000   
mean       0.492917       0.184747     39.620958        0.305590   
std        0.499959       0.388100     15.135685        0.460666   
min        0.000000       0.000000      7.000000        0.000000   
25%        0.000000       0.000000     27.000000        0.000000   
50%        0.000000       0.000000     40.000000        0.000000   
75%        1.000000       0.000000     51.000000        1.000000   
max        1.000000       1.000000     85.000000        1.000000   

              Class  Flight Distance  Inflight wifi service  \
count  25976.000000     25976.000000           25976.000000   
mean       0.592778      1193.788459               2.724746   
std        0.623703       998.683999               1.335384   
min        0.000000        31.000000               0.000000   
25%        0.000000       414.000000               2.000000   
50%      

<h1> Model (Pre-Training)

In [64]:
clf1 = SGDClassifier(loss="squared_error", random_state=1)
clf2 = RandomForestClassifier(n_estimators=100, random_state=1,max_depth=21)

<h1> Voting using Sci-kit Learn 

In [65]:
eclf1 = VotingClassifier(estimators=[
        ('SGD', clf1), ('Random Forest', clf2)], voting='hard')
eclf1 = eclf1.fit(training_x, training_y)

c:\Users\chanj\anaconda3\envs\tf2\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\chanj\anaconda3\envs\tf2\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


<h1>Scoring</h1>


In [66]:
cross_val_score(eclf1, test_x, test_y, cv=5, scoring='f1', n_jobs=-1)


array([0.85880077, 0.94431971, 0.9107308 , 0.93530997, 0.95174561])